##### Copyright 2019 The TensorFlow Authors.

In [4]:
#v: alles gleich aber iris daten

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import datetime
from sklearn.model_selection import train_test_split
import time
from sklearn.preprocessing import StandardScaler
import numpy as np # for math and arrays
import pandas as pd
from tqdm import tqdm
all_ds = pd.read_csv('iris_dataset.csv')
# all_ds = all_ds.sample(frac=1) # This will randomly shuffle the rows to make sure the data is not sorted
train_dataset, temp_test_dataset =  train_test_split(all_ds, test_size=0.3,random_state=42)
test_dataset, valid_dataset =  train_test_split(temp_test_dataset, test_size=0.5,random_state=42)
train_labels1 = train_dataset.pop('target')
test_labels1 = test_dataset.pop('target')
valid_labels1 = valid_dataset.pop('target')
train_labels = pd.get_dummies(train_labels1, prefix='Label')
valid_labels = pd.get_dummies(valid_labels1, prefix='Label')
test_labels = pd.get_dummies(test_labels1, prefix='Label')
train_stats = train_dataset.describe()
train_stats = train_stats.transpose()
normed_train_data = pd.DataFrame(StandardScaler().fit_transform(train_dataset), columns=train_dataset.columns, index=train_dataset.index)
normed_test_data = pd.DataFrame(StandardScaler().fit_transform(test_dataset), columns=test_dataset.columns, index=test_dataset.index)
normed_valid_data = pd.DataFrame(StandardScaler().fit_transform(valid_dataset), columns=valid_dataset.columns, index=valid_dataset.index)
x_train, y_train, x_valid, y_valid = normed_train_data, train_labels, normed_valid_data, valid_labels

HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32,64,128,256, 256*2,256*4,256*8,256*8*2*2*2]))
# HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
HP_LR = hp.HParam('lr', hp.Discrete([0.01,0.001,0.0001,0.00001,]))
METRIC_ACCURACY = 'accuracy'

log_name = 'logs_'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+'/hparam_tuning'

with tf.summary.create_file_writer(log_name).as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_OPTIMIZER, HP_LR],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

def helper_fct_return_optimizer_w_learn_rate(opt_name:str,lr:float):
  if opt_name == "sgd":
    return tf.keras.optimizers.SGD(learning_rate=lr)
  if opt_name == "adam":
    return tf.keras.optimizers.Adam(learning_rate=lr)
  return "error"

def train_test_model(hparams):
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten( input_shape = (4,)),
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS]),
    # tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    tf.keras.layers.Dense(3, activation='softmax'),
  ])
  
  model.compile(
      optimizer=helper_fct_return_optimizer_w_learn_rate(hparams[HP_OPTIMIZER],hparams[HP_LR],),
      loss='categorical_crossentropy',
      metrics=['accuracy'],
  )
  model.fit(x_train, y_train,validation_data=(x_valid,y_valid),epochs=100, shuffle=True,  verbose=False) 
  _, accuracy = model.evaluate(x_valid, y_valid,verbose=False)
  return accuracy

def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

session_num = 0
for num_units in tqdm(HP_NUM_UNITS.domain.values):
  for lr in HP_LR.domain.values:
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {HP_NUM_UNITS: num_units, HP_OPTIMIZER: optimizer, HP_LR: lr}
      run_name = "run-%d" % session_num
      run(log_name+ run_name, hparams)
      session_num += 1

100%|██████████| 9/9 [11:53<00:00, 79.27s/it]
